# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 5 2023 8:30AM,254301,10,8990139-SD,Eywa Pharma Inc.,Released
1,Jan 5 2023 8:30AM,254306,10,8990148-SD,Eywa Pharma Inc.,Released
2,Jan 5 2023 8:30AM,254310,10,8990153-SD,Eywa Pharma Inc.,Released
3,Jan 4 2023 6:15PM,254315,21,637716,"NBTY Global, Inc.",Released
4,Jan 4 2023 4:43PM,254312,15,PNC412001A,"Person & Covey, Inc.",Released
5,Jan 4 2023 4:43PM,254312,15,PNC412002A,"Person & Covey, Inc.",Released
6,Jan 4 2023 4:43PM,254312,15,PNC412003A,"Person & Covey, Inc.",Released
7,Jan 4 2023 4:43PM,254312,15,PNC412007A,"Person & Covey, Inc.",Released
8,Jan 4 2023 4:43PM,254312,15,PNC412008A,"Person & Covey, Inc.",Released
9,Jan 4 2023 4:43PM,254312,15,PNC412009A,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,254306,Released,1
13,254307,Released,6
14,254310,Released,1
15,254312,Released,36
16,254315,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
254306,NaN,1.0
254307,NaN,6.0
254310,NaN,1.0
254312,NaN,36.0
254315,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
254196,0.0,17.0
254255,1.0,0.0
254265,0.0,1.0
254266,0.0,1.0
254270,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
254196,0,17
254255,1,0
254265,0,1
254266,0,1
254270,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,254196,0,17
1,254255,1,0
2,254265,0,1
3,254266,0,1
4,254270,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,254196,,17
1,254255,1,
2,254265,,1
3,254266,,1
4,254270,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 5 2023 8:30AM,254301,10,Eywa Pharma Inc.
1,Jan 5 2023 8:30AM,254306,10,Eywa Pharma Inc.
2,Jan 5 2023 8:30AM,254310,10,Eywa Pharma Inc.
3,Jan 4 2023 6:15PM,254315,21,"NBTY Global, Inc."
4,Jan 4 2023 4:43PM,254312,15,"Person & Covey, Inc."
40,Jan 4 2023 3:56PM,254307,19,"AdvaGen Pharma, Ltd"
46,Jan 4 2023 3:52PM,254305,19,"Emersa Waterbox, LLC"
47,Jan 4 2023 3:48PM,254304,19,"GUSA Granules USA, Inc."
48,Jan 4 2023 3:36PM,254302,15,"Virtus Pharmaceuticals, LLC"
50,Jan 4 2023 3:30PM,254297,19,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jan 5 2023 8:30AM,254301,10,Eywa Pharma Inc.,,1
1,Jan 5 2023 8:30AM,254306,10,Eywa Pharma Inc.,,1
2,Jan 5 2023 8:30AM,254310,10,Eywa Pharma Inc.,,1
3,Jan 4 2023 6:15PM,254315,21,"NBTY Global, Inc.",,1
4,Jan 4 2023 4:43PM,254312,15,"Person & Covey, Inc.",,36
5,Jan 4 2023 3:56PM,254307,19,"AdvaGen Pharma, Ltd",,6
6,Jan 4 2023 3:52PM,254305,19,"Emersa Waterbox, LLC",,1
7,Jan 4 2023 3:48PM,254304,19,"GUSA Granules USA, Inc.",,1
8,Jan 4 2023 3:36PM,254302,15,"Virtus Pharmaceuticals, LLC",,2
9,Jan 4 2023 3:30PM,254297,19,"Emersa Waterbox, LLC",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 5 2023 8:30AM,254301,10,Eywa Pharma Inc.,1,
1,Jan 5 2023 8:30AM,254306,10,Eywa Pharma Inc.,1,
2,Jan 5 2023 8:30AM,254310,10,Eywa Pharma Inc.,1,
3,Jan 4 2023 6:15PM,254315,21,"NBTY Global, Inc.",1,
4,Jan 4 2023 4:43PM,254312,15,"Person & Covey, Inc.",36,
5,Jan 4 2023 3:56PM,254307,19,"AdvaGen Pharma, Ltd",6,
6,Jan 4 2023 3:52PM,254305,19,"Emersa Waterbox, LLC",1,
7,Jan 4 2023 3:48PM,254304,19,"GUSA Granules USA, Inc.",1,
8,Jan 4 2023 3:36PM,254302,15,"Virtus Pharmaceuticals, LLC",2,
9,Jan 4 2023 3:30PM,254297,19,"Emersa Waterbox, LLC",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 5 2023 8:30AM,254301,10,Eywa Pharma Inc.,1,
1,Jan 5 2023 8:30AM,254306,10,Eywa Pharma Inc.,1,
2,Jan 5 2023 8:30AM,254310,10,Eywa Pharma Inc.,1,
3,Jan 4 2023 6:15PM,254315,21,"NBTY Global, Inc.",1,
4,Jan 4 2023 4:43PM,254312,15,"Person & Covey, Inc.",36,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jan 5 2023 8:30AM,254301,10,Eywa Pharma Inc.,1.0,NaN
1,Jan 5 2023 8:30AM,254306,10,Eywa Pharma Inc.,1.0,NaN
2,Jan 5 2023 8:30AM,254310,10,Eywa Pharma Inc.,1.0,NaN
3,Jan 4 2023 6:15PM,254315,21,"NBTY Global, Inc.",1.0,NaN
4,Jan 4 2023 4:43PM,254312,15,"Person & Covey, Inc.",36.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 5 2023 8:30AM,254301,10,Eywa Pharma Inc.,1.0,0.0
1,Jan 5 2023 8:30AM,254306,10,Eywa Pharma Inc.,1.0,0.0
2,Jan 5 2023 8:30AM,254310,10,Eywa Pharma Inc.,1.0,0.0
3,Jan 4 2023 6:15PM,254315,21,"NBTY Global, Inc.",1.0,0.0
4,Jan 4 2023 4:43PM,254312,15,"Person & Covey, Inc.",36.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1017113,20.0,0.0
15,762884,39.0,0.0
16,254265,1.0,0.0
19,1525750,23.0,1.0
21,508581,2.0,0.0
22,254291,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1017113,20.0,0.0
1,15,762884,39.0,0.0
2,16,254265,1.0,0.0
3,19,1525750,23.0,1.0
4,21,508581,2.0,0.0
5,22,254291,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,20.0,0.0
1,15,39.0,0.0
2,16,1.0,0.0
3,19,23.0,1.0
4,21,2.0,0.0
5,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,20.0
1,15,Released,39.0
2,16,Released,1.0
3,19,Released,23.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21,22
Status,,,,,,
Executing,0.0,0.0,0.0,1.0,0.0,0.0
Released,20.0,39.0,1.0,23.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21,22
0,Executing,0.0,0.0,0.0,1.0,0.0,0.0
1,Released,20.0,39.0,1.0,23.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21,22
0,Executing,0.0,0.0,0.0,1.0,0.0,0.0
1,Released,20.0,39.0,1.0,23.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()